#  Modelo de ZeeHB

Sample of notebook for specific model

In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import pandas as pd
import numpy as np
import os, sys, inspect
import commands
from hep import *

In [3]:
import hep as hp
def Kappa(s2phi,M1,M2):
    return s2phi*np.log(M2**2/M1**2)/(4.*np.pi)**2

def Inverse_Zee_Matrix(kappa,ml,IH=False,sgnm2=-1,sgnml=1):
    """Inverse neutrino mass matrix normalized by kappa from
        Mnu_diag=U^T.(kappa Y).U
        ,
        (Inverse_Zee_Matrix) = U. Mnu_diag.U^T/kappa
        sgnm2=-1 guarantees real Yukawa couplings
        
    requires hep.py:
       https://github.com/restrepo/BSM-Toolbox/blob/master/tests/hep.py
    """
    if not IH:
        mltmp,Dm21_2,Dm3l_2,theta12,theta23,theta13,delta=hp.neutrino_data()
        U=hp.UPMNS(theta12[1],theta13[1],theta23[1])
        m1=ml; m2=sgnm2*np.sqrt(Dm21_2[1]+m1**2); m3=sgnml*np.sqrt(Dm3l_2[1]+m1**2)
    else:
        mltmp,Dm21_2,Dm3l_2,theta12,theta23,theta13,delta=hp.neutrino_data(IH=True)
        U=hp.UPMNS(theta12[1],theta13[1],theta23[1])
        #DEBUG signs
        m3=ml; m2=sgnm2*np.sqrt(Dm3l_2[1]-m3**2); m1=signml*np.sqrt(Dm21_2[1]-m2**2)
        
    return np.dot( np.dot( U,np.diag([m1,m2,m3]) ), U.transpose() )/kappa

def get_yukawas(X,q01=1e-3,q02=1e-6,q10=200e-3,q12=0,q21=-1e-5,q22=0,m_e=0.5109989461e-3,m_mu=0.1056583745,m_tau=1.77686):
    """
    Get the O and f full Yukawa mass matrices which define the neutrino mass matrix in the Zee model
      Mnu=kappa*(O.M_lep.f^T+f.M_lep.O^T)
    where M_lep is the diagonal matrix with the charged leptons
    
    The input X is the output of the `def Inverse_Zee_Matrix(...)` in this module
    """
    import numpy.lib.scimath as sc # .sqrt -> returns complex for sqrt(negative real)
    ml=np.array([m_e,m_mu,m_tau])
    Q=np.zeros((3,3))
    f=np.zeros((3,3))
    Q[0,1]=q01;Q[0,2]=q02;Q[1,0]=q10;Q[1,2]=q12;Q[2,1]=q21;Q[2,2]=q22
    Q[2,0]=( -(ml[0]**3*Q[1,0]**2*X[0,0]**2*(ml[1]*Q[2,1]*X[0,1] + ml[2]*Q[0,2]*X[2,2])) -\
      ml[0]*(ml[1]*Q[2,1]*X[1,1] + ml[2]*Q[1,2]*X[2,2])*(ml[1]*Q[0,1]*((ml[1]*Q[0,1]*X[0,1] +\
      ml[2]*Q[0,2]*X[0,2])*X[1,1] + ml[2]*Q[1,2]*X[0,0]*X[1,2]) +\
      ml[2]**2*Q[0,2]*Q[1,2]*X[0,0]*X[2,2]) + ml[0]**2*Q[1,0]*X[0,0]*(-(ml[1]*Q[2,1]*((2*ml[1]*Q[0,1]*X[0,1] +\
      ml[2]*Q[0,2]*X[0,2])*X[1,1] + ml[2]*Q[1,2]*X[0,0]*X[1,2])) -\
      ml[2]*(2*ml[2]*Q[0,2]*Q[1,2]*X[0,2] + ml[1]*Q[0,1]*(Q[1,2]*X[0,1] + Q[0,2]*X[1,1]))*X[2,2]) +\
      np.sqrt( -(ml[0]**2*ml[1]**2*(ml[0]*Q[1,0]*Q[2,1]*X[0,0] + Q[0,1]*(ml[1]*Q[2,1]*X[1,1] +\
      ml[2]*Q[1,2]*X[2,2]))**2*(ml[0]**2*Q[1,0]**2*X[0,0]**2*(-X[0,1]**2 + X[0,0]*X[1,1]) +\
      ml[1]**2*Q[0,1]**2*X[1,1]**2*(-X[0,1]**2 + X[0,0]*X[1,1]) +\
      2*ml[1]*ml[2]*Q[0,1]*X[1,1]*((Q[1,2]*X[0,0] - Q[0,2]*X[0,1])*X[0,2]*X[1,1] + X[0,0]*(-(Q[1,2]*X[0,1]) +\
      Q[0,2]*X[1,1])*X[1,2]) + 2*ml[0]*Q[1,0]*X[0,0]*(X[1,1]*(ml[2]*(Q[1,2]*X[0,0] - Q[0,2]*X[0,1])*X[0,2] +\
      ml[1]*Q[0,1]*(-X[0,1]**2 + X[0,0]*X[1,1])) + ml[2]*X[0,0]*(-(Q[1,2]*X[0,1]) + Q[0,2]*X[1,1])*X[1,2]) +\
      ml[2]**2*(-(Q[0,2]*X[0,2]*X[1,1] - Q[1,2]*X[0,0]*X[1,2])**2 + X[0,0]*X[1,1]*(Q[1,2]**2*X[0,0] -\
      2*Q[0,2]*Q[1,2]*X[0,1] + Q[0,2]**2*X[1,1])*X[2,2])))    )   )/\
      (ml[0]**2*X[0,0]*((ml[0]*Q[1,0]*X[0,0] + ml[1]*Q[0,1]*X[1,1])*(ml[0]*Q[1,0]*X[0,0] +\
       2*ml[2]*Q[1,2]*X[0,2] + ml[1]*Q[0,1]*X[1,1]) + ml[2]**2*Q[1,2]**2*X[0,0]*X[2,2]))

    Q[0,0]=(ml[0]**2*Q[0,1]*Q[1,0]*Q[2,0]*X[0,0] - ml[2]*(Q[0,2]*Q[2,1] -\
        Q[0,1]*Q[2,2])*(ml[1]*Q[2,1]*X[1,1] + ml[2]*Q[1,2]*X[2,2]) +\
        ml[0]*(ml[1]*Q[0,1]**2*Q[2,0]*X[1,1] + ml[2]*(Q[1,2]*Q[2,0]*(-(Q[2,1]*X[0,0]) +\
        2*Q[0,1]*X[0,2]) + Q[1,0]*(Q[2,1]*(Q[2,2]*X[0,0] - 2*Q[0,2]*X[0,2]) +\
        Q[0,1]*Q[0,2]*X[2,2]))))/(ml[0]*(ml[0]*Q[1,0]*Q[2,1]*X[0,0] +\
        Q[0,1]*(ml[1]*Q[2,1]*X[1,1] + ml[2]*Q[1,2]*X[2,2])))
    
    Q[1,1]=(ml[0]**3*Q[1,0]*Q[2,0]*(Q[0,1]*Q[1,2]*Q[2,0] - Q[0,2]*Q[1,0]*Q[2,1])*X[0,0]**2 +\
        ml[0]*(2*ml[1]*Q[0,1]*(Q[0,1]*Q[1,2]*Q[2,0]*X[0,1] + Q[1,0]*Q[2,1]*(Q[1,2]*X[0,0] -\
        Q[0,2]*X[0,1])) + ml[2]*(Q[0,1]*Q[1,2]*Q[2,0]*Q[2,2]*X[0,0] -\
        2*Q[0,2]**2*Q[1,0]*Q[2,1]*X[0,2] + Q[0,2]*(Q[1,0]*Q[2,1]*Q[2,2]*X[0,0] +\
        2*Q[1,2]*Q[2,0]*(-(Q[2,1]*X[0,0]) + Q[0,1]*X[0,2]))))*(ml[1]*Q[2,1]*X[1,1] +\
        ml[2]*Q[1,2]*X[2,2]) + (ml[1]*Q[0,1]**2*Q[1,2] + ml[2]*Q[0,2]*(-(Q[0,2]*Q[2,1]) +\
        Q[0,1]*Q[2,2]))*(ml[1]*Q[2,1]*X[1,1] + ml[2]*Q[1,2]*X[2,2])**2 +\
        ml[0]**2*X[0,0]*(ml[1]*(Q[1,0]**2*Q[2,1]**2*(Q[1,2]*X[0,0] - 2*Q[0,2]*X[0,1]) +\
        Q[0,1]**2*Q[1,2]*Q[2,0]**2*X[1,1] + Q[0,1]*Q[1,0]*Q[2,0]*Q[2,1]*(2*Q[1,2]*X[0,1] -\
        Q[0,2]*X[1,1])) + ml[2]*(Q[1,2]**2*Q[2,0]**2*(-(Q[2,1]*X[0,0]) + 2*Q[0,1]*X[0,2]) -\
        Q[0,2]**2*Q[1,0]**2*Q[2,1]*X[2,2] + Q[1,0]*Q[1,2]*Q[2,0]*(Q[2,1]*(Q[2,2]*X[0,0] -\
        2*Q[0,2]*X[0,2]) + Q[0,1]*Q[0,2]*X[2,2]))))/\
        (ml[1]*(ml[0]*Q[1,0]*Q[2,1]*X[0,0] + Q[0,1]*(ml[1]*Q[2,1]*X[1,1] +\
         ml[2]*Q[1,2]*X[2,2]))*(ml[0]*Q[1,2]*Q[2,0]*X[0,0] + Q[0,2]*(ml[1]*Q[2,1]*X[1,1] +\
         ml[2]*Q[1,2]*X[2,2])))    
        
    f[0,1]=(ml[0]*Q[1,2]*Q[2,0]*X[0,0] + ml[1]*Q[0,2]*Q[2,1]*X[1,1] + ml[2]*Q[0,2]*Q[1,2]*X[2,2])/\
       (2*ml[0]*ml[1]*Q[0,1]*Q[1,2]*Q[2,0] - 2*ml[0]*ml[1]*Q[0,2]*Q[1,0]*Q[2,1])
    f[0,2]=-((ml[0]*Q[1,0]*Q[2,1]*X[0,0] + ml[1]*Q[0,1]*Q[2,1]*X[1,1] + ml[2]*Q[0,1]*Q[1,2]*X[2,2])/\
        (2*ml[0]*ml[2]*Q[0,1]*Q[1,2]*Q[2,0] - 2*ml[0]*ml[2]*Q[0,2]*Q[1,0]*Q[2,1]))
    f[1,2]=(ml[0]*Q[1,0]*Q[2,0]*X[0,0] + ml[1]*Q[0,1]*Q[2,0]*X[1,1] + ml[2]*Q[0,2]*Q[1,0]*X[2,2])/\
       (2*ml[1]*ml[2]*Q[0,1]*Q[1,2]*Q[2,0] - 2*ml[1]*ml[2]*Q[0,2]*Q[1,0]*Q[2,1])
    f[1,0]=-f[0,1]; f[2,0]=-f[0,2]; f[2,1]=-f[1,2]
    
    return Q,f

def test_Zee():
    m_e=0.5109989461e-3;m_mu=0.1056583745;m_tau=1.77686
    mlep=np.diag([m_e,m_mu,m_tau])
    m1=0 # lightest neutrino
    s2phi=0.014; M1=200.; M2=300.
    X=Inverse_Zee_Matrix(Kappa(s2phi,M1,M2),m1)
    O,f=get_yukawas(X) #Use default input values
    Mnu=Kappa(s2phi,M1,M2)*( np.dot( np.dot(O,mlep),f.transpose() )+np.dot( np.dot(f,mlep),O.transpose()) )
    Mnu_diag,U=np.linalg.eig(Mnu)
    lo=np.argsort(np.abs(Mnu_diag))
    Mnu_diag=np.array([Mnu_diag[lo[0]],Mnu_diag[lo[1]],Mnu_diag[lo[2]]])
    U=np.matrix(U)
    U=np.asarray(np.hstack((U[:,lo[0]],U[:,lo[1]],U[:,lo[2]])))
    
    mltmp,Dm21_2,Dm3l_2,theta12,theta23,theta13,delta=hp.neutrino_data()
    m2=np.sqrt(Dm21_2[1]+m1**2); m3=np.sqrt(Dm3l_2[1]+m1**2)
    
    np.testing.assert_array_almost_equal(np.abs(U),\
        np.abs( hp.UPMNS(theta12[1],theta13[1],theta23[1]) ) )
    
    return np.testing.assert_almost_equal( np.abs(Mnu_diag),np.abs([m1,m2,m3]) ) 

In [4]:
test_Zee()

In [5]:
def func(s2phi, M1, M2, MAo, Mho, MHo, v, lam2, lam6, lam7, lam9, lam10, lamh, Muh2, Mu22):
    M12 = M1*M1; M22 = M2*M2; MAo2 = MAo*MAo; Mho2 = Mho*Mho; MHo2 = MHo*MHo
    k = s2phi*np.log(M22/M12)/(4.*np.pi)**2
    phi = 0.5*np.arcsin(s2phi)
    Mu  = (M22-M12)*s2phi/(np.sqrt(2.0)*v)
    MH2 = M12*(np.sin(phi))**2+M22*(np.cos(phi))**2
    M332 = M12*(np.cos(phi))**2+M22*(np.sin(phi))**2
    lam1 = 0.5*(MHo2+Mho2-np.sqrt((MHo2-Mho2)**2-4.0*v**4*lam6**2))/(v**2)
    lam3 = 2.0*(MH2-Mu22)/(v**2)
    lam4 = 0.5*(np.sqrt((MHo2-Mho2)**2-4.0*v**4*lam6**2)+MHo2+Mho2+2.0*(MAo2-2.0*MH2))/(v**2)
    lam5 = 0.5*(np.sqrt((MHo2-Mho2)**2-4.0*v**4*lam6**2)+MHo2+Mho2-2.0*MAo2)/(v**2)
    lam8 = 2.0*(M332-Muh2)/(v**2)
    return k, Mu, lam1, lam3, lam4, lam5, lam8


## Check one point

In [6]:
a=hep(MODEL='radinuZeeHB')

`a-object` is an object with many attributes and methods. Use the tab to explore them. Some of them are
* a.Series: [pandas](http://pandas.pydata.org/) Series object with the "relevant" variables 
* a.LHA: Input LesHouces file as [pyslha](https://pypi.python.org/pypi/pyslha/) object
* a.runSPheno() -> a.LHA_out: return LHA output files as [pyslha](https://pypi.python.org/pypi/pyslha/) object
* a.runmicromegas() -> a.runSPheno() -> Updated the `a-object`  with micrOMEGAS "relevant" output

In [7]:
v = a.vev
s2phi = 1.4E-02
M1 = 200.0
M2 = 300.0
MAo = 90.0
Mho = 125.0
MHo = 700.0
lam2 = 1.3000000E-01    # lambda2Input
lam6 = 1.000000E-03     # lambda6Inputd 
lam7 = 0.000000E+00     # lambda7Input
lam9 = 0.000000E+00     # lambda9Input
lam10 = 0.000000E+00    # lambda10Input
lamh = 0.000000E+00     # lambdahInput
Muh2 = 1.000000E+02**2      # MhInput
Mu22 = 2.0000000E+02**2     # mEt2Input
m_e=0.5109989461e-3;m_mu=0.1056583745;m_tau=1.77686
mlep=np.diag([m_e,m_mu,m_tau])
m1=0 

k, Mu, lam1, lam3, lam4, lam5, lam8 = func(s2phi, M1, M2, MAo, Mho, MHo, v, lam2, lam6, lam7, lam9, lam10, lamh, Muh2, Mu22)

X=Inverse_Zee_Matrix(k,m1)
O,f=get_yukawas(X)

devnull=commands.getoutput('rm -f SPheno.spc.%s' %a.MODEL)

a.LHA.blocks['SPHENOINPUT'].entries[55]='0               # Calculate one loop masses'
a.LHA.blocks['MINPAR'][1]= '%0.8E       #lambda1Input' %lam1
a.LHA.blocks['MINPAR'][2]= '%0.8E       #lambda2Input' %lam2
a.LHA.blocks['MINPAR'][3]= '%0.8E       #lambda3Input' %lam3
a.LHA.blocks['MINPAR'][4]= '%0.8E       #lambda4Input' %lam4
a.LHA.blocks['MINPAR'][5]= '%0.8E       #lambda5Input' %lam5
a.LHA.blocks['MINPAR'][6]= '%0.8E       #lambda6Input' %lam6
a.LHA.blocks['MINPAR'][7]= '%0.8E       #lambda7Input' %lam7
a.LHA.blocks['MINPAR'][8]= '%0.8E       #lambda8Input' %lam8
a.LHA.blocks['MINPAR'][9]= '%0.8E       #lambda9Input' %lam9
a.LHA.blocks['MINPAR'][10]='%0.8E       #lambda10Input'%lam10
a.LHA.blocks['MINPAR'][11]='%0.8E       #lambdahInput' %lamh
a.LHA.blocks['MINPAR'][12]='%0.8E       #MhInput' %Muh2
a.LHA.blocks['MINPAR'][13]='%0.8E       #MuInput' %Mu
a.LHA.blocks['MINPAR'][14]='%0.8E       #mEt2Input' %Mu22

for i in range(3):
    for j in range(3):
        a.LHA.blocks['YHIN'][i+1,j+1]='%0.8E      # Yh(%d,%d)'    %(f[i,j],i+1,j+1)
        a.LHA.blocks['EPSEIN'][i+1,j+1]='%0.8E      # epsE(%d,%d)'    %(O[i,j],i+1,j+1) # Note ordering
        
slha=a.runSPheno()

## Test

In [8]:
#Expected
Mnu=k*( np.dot( np.dot(O,mlep),f.transpose() )+np.dot( np.dot(f,mlep),O.transpose()) )
Mnu_diag,U=np.linalg.eig(Mnu)
lo=np.argsort(np.abs(Mnu_diag))
Mnu_diag=np.array([Mnu_diag[lo[0]],Mnu_diag[lo[1]],Mnu_diag[lo[2]]])
U=np.matrix(U)
U=np.asarray(np.hstack((U[:,lo[0]],U[:,lo[1]],U[:,lo[2]])))

#Obtained
Unew=np.zeros((3,3))
for i in range(3):
    for j in range(3):
        Unew[i,j]=a.LHA_out.blocks['UVMIX'].entries[i+1,j+1]
Mnu_diag_new=np.array([a.LHA_out.blocks['MASS'].entries[12],a.LHA_out.blocks['MASS'].entries[14],\
                      a.LHA_out.blocks['MASS'].entries[16]  ])

In [9]:
np.testing.assert_array_almost_equal(np.abs( Mnu_diag*1E11 ),np.abs( Mnu_diag_new*1E11 ),decimal=1)
np.testing.assert_array_almost_equal(np.abs(U.transpose()),np.abs(Unew),decimal=3)
np.testing.assert_array_almost_equal( np.array([Mho,MHo,MAo,M1,M2,s2phi]),\
    np.array([ a.LHA_out.blocks['MASS'].entries[25],\
    a.LHA_out.blocks['MASS'].entries[35],\
    a.LHA_out.blocks['MASS'].entries[36],\
    a.LHA_out.blocks['MASS'].entries[37],\
    a.LHA_out.blocks['MASS'].entries[900037],\
    np.sin(2*np.arcsin(a.LHA_out.blocks['CHARGEMIX'].entries[2,3])) ] ),decimal=4  )

In [10]:
pd.Series(a.LHA_out_with_comments.blocks['FLAVORKITLFV'].entries)

701            1.75081795E-05  # BR(mu->e gamma)
702           1.53702757E-08  # BR(tau->e gamma)
703          4.81067747E-10  # BR(tau->mu gamma)
800               1.20712919E-04  # CR(mu-e, Al)
801               2.21644746E-04  # CR(mu-e, Ti)
802               2.95652615E-04  # CR(mu-e, Sr)
803               3.49744067E-04  # CR(mu-e, Sb)
804               1.92671085E-04  # CR(mu-e, Au)
805               1.82257941E-04  # CR(mu-e, Pb)
901                 5.43527426E-02  # BR(mu->3e)
902                6.23982989E-05  # BR(tau->3e)
903               3.63482273E-10  # BR(tau->3mu)
904     8.27406442E-09  # BR(tau- -> e- mu+ mu-)
905      1.29523069E-06  # BR(tau- -> mu- e+ e-)
906     5.13106193E-13  # BR(tau- -> e+ mu- mu-)
907      6.07643813E-07  # BR(tau- -> mu+ e- e-)
1001               5.93605370E-08  # BR(Z->e mu)
1002              2.78433682E-10  # BR(Z->e tau)
1003             8.68218172E-12  # BR(Z->mu tau)
1101               2.23222251E-03  # BR(h->e mu)
1102              1.

## Rescale solution after $\kappa$ change
We now change $\kappa$ and after rescaling $O$ and $f$ we could have the new solution

We have
\begin{equation}
M_\nu=\kappa (O M_l\, f^T+f M_l\, O^T)
\end{equation}
We want rescale $\kappa\to \kappa'=a\kappa$ and $O,f\to O',f'=bO,cf$ such that $M_\nu$ remain the same
Therefore
\begin{align}
M_\nu\to M_\nu'=&\kappa' (O' M_l\, f^{\prime T}+f' M_l\, O^{\prime T})\\
=&abc\kappa (O M_l\, f^T+f M_l\, O^T)
\end{align}
The condition is fulfill if 
\begin{align}
b=c=\frac{1}{\sqrt{a}}=\sqrt{\frac{\kappa}{\kappa'}}
\end{align}

### Change $\sin(2\phi)$:

In [11]:
s2phi=0.1
kp, Mu, lam1, lam3, lam4, lam5, lam8 = func(s2phi, M1, M2, MAo, Mho, MHo, v, lam2, lam6, lam7, lam9, lam10, lamh, Muh2, Mu22)

devnull=commands.getoutput('rm -f SPheno.spc.%s' %a.MODEL)

a.LHA.blocks['SPHENOINPUT'].entries[55]='0               # Calculate one loop masses'
a.LHA.blocks['MINPAR'][1]= '%0.8E       #lambda1Input' %lam1
a.LHA.blocks['MINPAR'][2]= '%0.8E       #lambda2Input' %lam2
a.LHA.blocks['MINPAR'][3]= '%0.8E       #lambda3Input' %lam3
a.LHA.blocks['MINPAR'][4]= '%0.8E       #lambda4Input' %lam4
a.LHA.blocks['MINPAR'][5]= '%0.8E       #lambda5Input' %lam5
a.LHA.blocks['MINPAR'][6]= '%0.8E       #lambda6Input' %lam6
a.LHA.blocks['MINPAR'][7]= '%0.8E       #lambda7Input' %lam7
a.LHA.blocks['MINPAR'][8]= '%0.8E       #lambda8Input' %lam8
a.LHA.blocks['MINPAR'][9]= '%0.8E       #lambda9Input' %lam9
a.LHA.blocks['MINPAR'][10]='%0.8E       #lambda10Input'%lam10
a.LHA.blocks['MINPAR'][11]='%0.8E       #lambdahInput' %lamh
a.LHA.blocks['MINPAR'][12]='%0.8E       #MhInput' %Muh2
a.LHA.blocks['MINPAR'][13]='%0.8E       #MuInput' %Mu
a.LHA.blocks['MINPAR'][14]='%0.8E       #mEt2Input' %Mu22

for i in range(3):
    for j in range(3):
        a.LHA.blocks['YHIN'][i+1,j+1]='%0.8E      # Yh(%d,%d)'    %(np.sqrt(k/kp)*f[i,j],i+1,j+1)
        a.LHA.blocks['EPSEIN'][i+1,j+1]='%0.8E      # epsE(%d,%d)'    %(np.sqrt(k/kp)*O[i,j],i+1,j+1) # Note ordering
        
slha=a.runSPheno()

#### Test:

In [12]:
Unew=np.zeros((3,3))
for i in range(3):
    for j in range(3):
        Unew[i,j]=a.LHA_out.blocks['UVMIX'].entries[i+1,j+1]
Mnu_diag_new=np.array([a.LHA_out.blocks['MASS'].entries[12],a.LHA_out.blocks['MASS'].entries[14],\
                      a.LHA_out.blocks['MASS'].entries[16]  ])

np.testing.assert_array_almost_equal(np.abs( Mnu_diag*1E11 ),np.abs( Mnu_diag_new*1E11 ),decimal=1)
np.testing.assert_array_almost_equal(np.abs(U.transpose()),np.abs(Unew),decimal=3)
np.testing.assert_array_almost_equal( np.array([Mho,MHo,MAo,M1,M2,s2phi]),\
    np.array([ a.LHA_out.blocks['MASS'].entries[25],\
    a.LHA_out.blocks['MASS'].entries[35],\
    a.LHA_out.blocks['MASS'].entries[36],\
    a.LHA_out.blocks['MASS'].entries[37],\
    a.LHA_out.blocks['MASS'].entries[900037],\
    np.sin(2*np.arcsin(a.LHA_out.blocks['CHARGEMIX'].entries[2,3])) ] ),decimal=4  )

In [13]:
pd.Series(a.LHA_out_with_comments.blocks['FLAVORKITLFV'].entries)

701            3.43075701E-07  # BR(mu->e gamma)
702           3.01277997E-10  # BR(tau->e gamma)
703          9.42959222E-12  # BR(tau->mu gamma)
800               2.36618055E-06  # CR(mu-e, Al)
801               4.34464054E-06  # CR(mu-e, Ti)
802               5.79530736E-06  # CR(mu-e, Sr)
803               6.85568408E-06  # CR(mu-e, Sb)
804               3.77676071E-06  # CR(mu-e, Au)
805               3.57264619E-06  # CR(mu-e, Pb)
901                 1.01331219E-03  # BR(mu->3e)
902                1.16902058E-06  # BR(tau->3e)
903               7.03273213E-12  # BR(tau->3mu)
904     1.59905013E-10  # BR(tau- -> e- mu+ mu-)
905      2.45630973E-08  # BR(tau- -> mu- e+ e-)
906     1.00482170E-14  # BR(tau- -> e+ mu- mu-)
907      1.14435789E-08  # BR(tau- -> mu+ e- e-)
1001               1.16341838E-09  # BR(Z->e mu)
1002              5.45727968E-12  # BR(Z->e tau)
1003             1.70170085E-13  # BR(Z->mu tau)
1101               3.39253256E-04  # BR(h->e mu)
1102              2.

### Change $M_2$:

In [14]:
s2phi = 1.4E-02
M2 = 3000.0

kp, Mu, lam1, lam3, lam4, lam5, lam8 = func(s2phi, M1, M2, MAo, Mho, MHo, v, lam2, lam6, lam7, lam9, lam10, lamh, Muh2, Mu22)

devnull=commands.getoutput('rm -f SPheno.spc.%s' %a.MODEL)

a.LHA.blocks['SPHENOINPUT'].entries[55]='0               # Calculate one loop masses'
a.LHA.blocks['MINPAR'][1]= '%0.8E       #lambda1Input' %lam1
a.LHA.blocks['MINPAR'][2]= '%0.8E       #lambda2Input' %lam2
a.LHA.blocks['MINPAR'][3]= '%0.8E       #lambda3Input' %lam3
a.LHA.blocks['MINPAR'][4]= '%0.8E       #lambda4Input' %lam4
a.LHA.blocks['MINPAR'][5]= '%0.8E       #lambda5Input' %lam5
a.LHA.blocks['MINPAR'][6]= '%0.8E       #lambda6Input' %lam6
a.LHA.blocks['MINPAR'][7]= '%0.8E       #lambda7Input' %lam7
a.LHA.blocks['MINPAR'][8]= '%0.8E       #lambda8Input' %lam8
a.LHA.blocks['MINPAR'][9]= '%0.8E       #lambda9Input' %lam9
a.LHA.blocks['MINPAR'][10]='%0.8E       #lambda10Input'%lam10
a.LHA.blocks['MINPAR'][11]='%0.8E       #lambdahInput' %lamh
a.LHA.blocks['MINPAR'][12]='%0.8E       #MhInput' %Muh2
a.LHA.blocks['MINPAR'][13]='%0.8E       #MuInput' %Mu
a.LHA.blocks['MINPAR'][14]='%0.8E       #mEt2Input' %Mu22

for i in range(3):
    for j in range(3):
        a.LHA.blocks['YHIN'][i+1,j+1]='%0.8E      # Yh(%d,%d)'    %(np.sqrt(k/kp)*f[i,j],i+1,j+1)
        a.LHA.blocks['EPSEIN'][i+1,j+1]='%0.8E      # epsE(%d,%d)'    %(np.sqrt(k/kp)*O[i,j],i+1,j+1) # Note ordering
        
slha=a.runSPheno()

#### Test

In [15]:
Unew=np.zeros((3,3))
for i in range(3):
    for j in range(3):
        Unew[i,j]=a.LHA_out.blocks['UVMIX'].entries[i+1,j+1]
Mnu_diag_new=np.array([a.LHA_out.blocks['MASS'].entries[12],a.LHA_out.blocks['MASS'].entries[14],\
                      a.LHA_out.blocks['MASS'].entries[16]  ])

np.testing.assert_array_almost_equal(np.abs( Mnu_diag*1E11 ),np.abs( Mnu_diag_new*1E11 ),decimal=1)
np.testing.assert_array_almost_equal(np.abs(U.transpose()),np.abs(Unew),decimal=3)
np.testing.assert_array_almost_equal( np.array([Mho,MHo,MAo,M1,M2,s2phi]),\
    np.array([ a.LHA_out.blocks['MASS'].entries[25],\
    a.LHA_out.blocks['MASS'].entries[35],\
    a.LHA_out.blocks['MASS'].entries[36],\
    a.LHA_out.blocks['MASS'].entries[37],\
    a.LHA_out.blocks['MASS'].entries[900037],\
    np.sin(2*np.arcsin(a.LHA_out.blocks['CHARGEMIX'].entries[2,3])) ] ),decimal=4 )

In [16]:
pd.Series(a.LHA_out_with_comments.blocks['FLAVORKITLFV'].entries)

701            3.92532083E-07  # BR(mu->e gamma)
702           3.44563875E-10  # BR(tau->e gamma)
703          1.07843520E-11  # BR(tau->mu gamma)
800               2.70637245E-06  # CR(mu-e, Al)
801               4.96927539E-06  # CR(mu-e, Ti)
802               6.62851173E-06  # CR(mu-e, Sr)
803               7.84132015E-06  # CR(mu-e, Sb)
804               4.31973744E-06  # CR(mu-e, Au)
805               4.08627661E-06  # CR(mu-e, Pb)
901                 1.19007891E-03  # BR(mu->3e)
902                1.37266324E-06  # BR(tau->3e)
903               8.04500089E-12  # BR(tau->3mu)
904     1.82922309E-10  # BR(tau- -> e- mu+ mu-)
905      2.85025063E-08  # BR(tau- -> mu- e+ e-)
906     1.16947963E-14  # BR(tau- -> e+ mu- mu-)
907      1.34235202E-08  # BR(tau- -> mu+ e- e-)
1001               1.33095759E-09  # BR(Z->e mu)
1002              6.24163533E-12  # BR(Z->e tau)
1003             1.94627827E-13  # BR(Z->mu tau)
1101               7.68645380E-04  # BR(h->e mu)
1102              4.

Or just decoupling the scalars

In [20]:
s2phi = 1.4E-02
M2 = 5000.0
M1=3010
MAo=3010
MHo=3005
kp, Mu, lam1, lam3, lam4, lam5, lam8 = func(s2phi, M1, M2, MAo, Mho, MHo, v, lam2, lam6, lam7, lam9, lam10, lamh, Muh2, Mu22)

devnull=commands.getoutput('rm -f SPheno.spc.%s' %a.MODEL)

a.LHA.blocks['SPHENOINPUT'].entries[55]='0               # Calculate one loop masses'
a.LHA.blocks['MINPAR'][1]= '%0.8E       #lambda1Input' %lam1
a.LHA.blocks['MINPAR'][2]= '%0.8E       #lambda2Input' %lam2
a.LHA.blocks['MINPAR'][3]= '%0.8E       #lambda3Input' %lam3
a.LHA.blocks['MINPAR'][4]= '%0.8E       #lambda4Input' %lam4
a.LHA.blocks['MINPAR'][5]= '%0.8E       #lambda5Input' %lam5
a.LHA.blocks['MINPAR'][6]= '%0.8E       #lambda6Input' %lam6
a.LHA.blocks['MINPAR'][7]= '%0.8E       #lambda7Input' %lam7
a.LHA.blocks['MINPAR'][8]= '%0.8E       #lambda8Input' %lam8
a.LHA.blocks['MINPAR'][9]= '%0.8E       #lambda9Input' %lam9
a.LHA.blocks['MINPAR'][10]='%0.8E       #lambda10Input'%lam10
a.LHA.blocks['MINPAR'][11]='%0.8E       #lambdahInput' %lamh
a.LHA.blocks['MINPAR'][12]='%0.8E       #MhInput' %Muh2
a.LHA.blocks['MINPAR'][13]='%0.8E       #MuInput' %Mu
a.LHA.blocks['MINPAR'][14]='%0.8E       #mEt2Input' %Mu22

for i in range(3):
    for j in range(3):
        a.LHA.blocks['YHIN'][i+1,j+1]='%0.8E      # Yh(%d,%d)'    %(np.sqrt(k/kp)*f[i,j],i+1,j+1)
        a.LHA.blocks['EPSEIN'][i+1,j+1]='%0.8E      # epsE(%d,%d)'    %(np.sqrt(k/kp)*O[i,j],i+1,j+1) # Note ordering
        
slha=a.runSPheno()

#### Test

In [21]:
Unew=np.zeros((3,3))
for i in range(3):
    for j in range(3):
        Unew[i,j]=a.LHA_out.blocks['UVMIX'].entries[i+1,j+1]
Mnu_diag_new=np.array([a.LHA_out.blocks['MASS'].entries[12],a.LHA_out.blocks['MASS'].entries[14],\
                      a.LHA_out.blocks['MASS'].entries[16]  ])

np.testing.assert_array_almost_equal(np.abs( Mnu_diag*1E11 ),np.abs( Mnu_diag_new*1E11 ),decimal=1)
np.testing.assert_array_almost_equal(np.abs(U.transpose()),np.abs(Unew),decimal=3)
np.testing.assert_array_almost_equal( np.array([Mho,MHo,MAo,M1,M2,s2phi]),\
    np.array([ a.LHA_out.blocks['MASS'].entries[25],\
    a.LHA_out.blocks['MASS'].entries[35],\
    a.LHA_out.blocks['MASS'].entries[36],\
    a.LHA_out.blocks['MASS'].entries[37],\
    a.LHA_out.blocks['MASS'].entries[900037],\
    np.sin(2*np.arcsin(a.LHA_out.blocks['CHARGEMIX'].entries[2,3])) ] ),decimal=4 )

In [22]:
pd.Series(a.LHA_out_with_comments.blocks['FLAVORKITLFV'].entries)

701            5.71416024E-11  # BR(mu->e gamma)
702           3.75173912E-14  # BR(tau->e gamma)
703          1.17184450E-15  # BR(tau->mu gamma)
800               4.34766026E-10  # CR(mu-e, Al)
801               7.83036302E-10  # CR(mu-e, Ti)
802               1.05928219E-09  # CR(mu-e, Sr)
803               1.19360426E-09  # CR(mu-e, Sb)
804               6.43478655E-10  # CR(mu-e, Au)
805               6.05395434E-10  # CR(mu-e, Pb)
901                 8.83495117E-08  # BR(mu->3e)
902                9.79076366E-11  # BR(tau->3e)
903               1.70513087E-15  # BR(tau->3mu)
904     3.39245071E-14  # BR(tau- -> e- mu+ mu-)
905      3.10462846E-12  # BR(tau- -> mu- e+ e-)
906     3.08602362E-18  # BR(tau- -> e+ mu- mu-)
907      4.62911388E-16  # BR(tau- -> mu+ e- e-)
1001               4.72643510E-13  # BR(Z->e mu)
1002              3.87150959E-16  # BR(Z->e tau)
1003             1.18457426E-17  # BR(Z->mu tau)
1101               2.92857766E-02  # BR(h->e mu)
1102              1.

### Chaging the condition
We could chek if changing the  condition helps with the suppression of some observables. For example:

\begin{align}
b=&1 &c=&\frac{1}{a}=\frac{\kappa}{\kappa'}
\end{align}
or
\begin{align}
c=&1 &b=&\frac{1}{a}=\frac{\kappa}{\kappa'}
\end{align}

In [29]:
s2phi = 1.4E-02
M2 = 5000.0
M1=3010
MAo=3010
MHo=3005
kp, Mu, lam1, lam3, lam4, lam5, lam8 = func(s2phi, M1, M2, MAo, Mho, MHo, v, lam2, lam6, lam7, lam9, lam10, lamh, Muh2, Mu22)

devnull=commands.getoutput('rm -f SPheno.spc.%s' %a.MODEL)

a.LHA.blocks['SPHENOINPUT'].entries[55]='0               # Calculate one loop masses'
a.LHA.blocks['MINPAR'][1]= '%0.8E       #lambda1Input' %lam1
a.LHA.blocks['MINPAR'][2]= '%0.8E       #lambda2Input' %lam2
a.LHA.blocks['MINPAR'][3]= '%0.8E       #lambda3Input' %lam3
a.LHA.blocks['MINPAR'][4]= '%0.8E       #lambda4Input' %lam4
a.LHA.blocks['MINPAR'][5]= '%0.8E       #lambda5Input' %lam5
a.LHA.blocks['MINPAR'][6]= '%0.8E       #lambda6Input' %lam6
a.LHA.blocks['MINPAR'][7]= '%0.8E       #lambda7Input' %lam7
a.LHA.blocks['MINPAR'][8]= '%0.8E       #lambda8Input' %lam8
a.LHA.blocks['MINPAR'][9]= '%0.8E       #lambda9Input' %lam9
a.LHA.blocks['MINPAR'][10]='%0.8E       #lambda10Input'%lam10
a.LHA.blocks['MINPAR'][11]='%0.8E       #lambdahInput' %lamh
a.LHA.blocks['MINPAR'][12]='%0.8E       #MhInput' %Muh2
a.LHA.blocks['MINPAR'][13]='%0.8E       #MuInput' %Mu
a.LHA.blocks['MINPAR'][14]='%0.8E       #mEt2Input' %Mu22

for i in range(3):
    for j in range(3):
        a.LHA.blocks['YHIN'][i+1,j+1]='%0.8E      # Yh(%d,%d)'    %(f[i,j],i+1,j+1)
        a.LHA.blocks['EPSEIN'][i+1,j+1]='%0.8E      # epsE(%d,%d)'    %((k/kp)*O[i,j],i+1,j+1) # Note ordering
        
slha=a.runSPheno()

#### Test

In [30]:
Unew=np.zeros((3,3))
for i in range(3):
    for j in range(3):
        Unew[i,j]=a.LHA_out.blocks['UVMIX'].entries[i+1,j+1]
Mnu_diag_new=np.array([a.LHA_out.blocks['MASS'].entries[12],a.LHA_out.blocks['MASS'].entries[14],\
                      a.LHA_out.blocks['MASS'].entries[16]  ])

np.testing.assert_array_almost_equal(np.abs( Mnu_diag*1E11 ),np.abs( Mnu_diag_new*1E11 ),decimal=1)
np.testing.assert_array_almost_equal(np.abs(U.transpose()),np.abs(Unew),decimal=3)
np.testing.assert_array_almost_equal( np.array([Mho,MHo,MAo,M1,M2,s2phi]),\
    np.array([ a.LHA_out.blocks['MASS'].entries[25],\
    a.LHA_out.blocks['MASS'].entries[35],\
    a.LHA_out.blocks['MASS'].entries[36],\
    a.LHA_out.blocks['MASS'].entries[37],\
    a.LHA_out.blocks['MASS'].entries[900037],\
    np.sin(2*np.arcsin(a.LHA_out.blocks['CHARGEMIX'].entries[2,3])) ] ),decimal=4 )

In [31]:
pd.Series(a.LHA_out_with_comments.blocks['FLAVORKITLFV'].entries)

701            4.50399732E-11  # BR(mu->e gamma)
702           2.23788553E-14  # BR(tau->e gamma)
703          6.97416809E-16  # BR(tau->mu gamma)
800               2.79698792E-10  # CR(mu-e, Al)
801               5.03763681E-10  # CR(mu-e, Ti)
802               6.81474098E-10  # CR(mu-e, Sr)
803               7.67933159E-10  # CR(mu-e, Sb)
804               4.14007958E-10  # CR(mu-e, Au)
805               3.89508192E-10  # CR(mu-e, Pb)
901                 5.70936242E-08  # BR(mu->3e)
902                6.28979781E-11  # BR(tau->3e)
903               1.10188780E-15  # BR(tau->3mu)
904     2.12084208E-14  # BR(tau- -> e- mu+ mu-)
905      2.02252828E-12  # BR(tau- -> mu- e+ e-)
906     1.42583431E-18  # BR(tau- -> e+ mu- mu-)
907      2.86090736E-16  # BR(tau- -> mu+ e- e-)
1001               5.07347893E-13  # BR(Z->e mu)
1002              1.62565531E-16  # BR(Z->e tau)
1003             4.87795648E-18  # BR(Z->mu tau)
1101               2.96725188E-02  # BR(h->e mu)
1102              1.

[![Home](http://www.incredimail.com/images/nav%20bar/home-icon.png)](./) 
[Jupyter home](./draft.pdf)